<a href="https://colab.research.google.com/github/navya-1122/Vision-Group-Internship/blob/main/cifar_10_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),      #Converts images from [0, 255] range to [0, 1] tensors
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])        #Shifts pixel values to [-1, 1]
batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,          #Downloads CIFAR-10 training set if not already there
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0)  #Loads the test set

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     #Unnormalizes the image from [-1, 1] back to [0, 1] range
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)      #Creates an iterator from the training data loader
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))   #Converts numeric labels to class names and prints them

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):  #Defines a neural network class that inherits from nn.Module
    def __init__(self):
        super().__init__()   #initializes the base class
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)  #a max pooling layer reduces the spatial dimensions of the input by taking the maximum value in each 2x2 region
        self.conv2 = nn.Conv2d(6, 16, 5)   #defines the first convolutional layer with 3 input channels (RGB), 6 output channels, and a kernel size of 5
        self.fc1 = nn.Linear(16 * 5 * 5, 120)    #first dense layer
        self.fc2 = nn.Linear(120, 84)       #second dense layer
        self.fc3 = nn.Linear(84, 10)        #third dense layer

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))       #conv1 + ReLU + pooling
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))     #fc1 + ReLU
        x = F.relu(self.fc2(x))
        x = self.fc3(x)   #output layer
        return x


model = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss() #Defines the loss function as CrossEntropyLoss, which is suitable for multi-class classification tasks.
# It combines softmax activation and negative log likelihood loss in one step.
optimizer = optim.Adam(model.parameters(), lr=0.001) #optimizer is used to update the model's parameters based on the computed gradients.
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1) #scheduler is used to adjust the learning rate during training. StepLR reduces the learning rate by a factor of gamma every step_size epochs.

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')      #CUDA lets programs use a GPU to run many calculations at the same time for faster processing
model = Net().to(device)   #Moves the model to the specified device (GPU or CPU) for training

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



num_epochs = 60
train_loss_history = []
train_acc_history = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in trainloader:  #Iterates over the training data loader to get batches of inputs and labels
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  #Resets the gradients of the model parameters to zero before computing the gradients for the current batch
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()  #Computes the gradients of the loss with respect to the model parameters
        optimizer.step()  #Updates the model parameters using the computed gradients

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(trainloader)
    epoch_acc = 100 * correct / total
    train_loss_history.append(epoch_loss)
    train_acc_history.append(epoch_acc)

    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc:.2f}%")


In [ ]:
PATH = './cifar_net.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
dataiter = iter(testloader)   #Creates an iterator from the test data loader
images, labels = next(dataiter)    #Gets the next batch of images and labels from the test set

# print images
imshow(torchvision.utils.make_grid(images))   #Displays a grid of images from the test set
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

In [ ]:
model = Net()
model.load_state_dict(torch.load(PATH))    #Loads the saved model parameters from the specified path into the new model instance

In [ ]:
outputs = model(images)

In [ ]:
_, predicted = torch.max(outputs, 1)   #Gets the predicted class labels by taking the index of the maximum output value for each image

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'    #Formats and prints the predicted class labels for display
                              for j in range(4)))

In [ ]:
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

In [ ]:
model.to(device)   #moves the neural network model to run on the specified device

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

# Loss Plot
plt.subplot(1, 2, 1)
plt.plot(train_loss_history, label='Training Loss', color='red')
plt.title('Training Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

# Accuracy Plot
plt.subplot(1, 2, 2)
plt.plot(train_acc_history, label='Training Accuracy', color='blue')
plt.title('Training Accuracy per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
inputs, labels = data[0].to(device), data[1].to(device)

In [ ]:
def show_confusion_matrix(model, dataloader, classes, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
    import matplotlib.pyplot as plt

    cm = confusion_matrix(all_labels, all_preds)

    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    disp.plot(cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

show_confusion_matrix(model, testloader, classes, device)


In [ ]:
from sklearn.metrics import classification_report

# Collect true labels and predictions from testloader
all_labels = []
all_preds = []

model.eval()
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Generate and print classification report using your classes names
report = classification_report(all_labels, all_preds, target_names=classes, digits=4)
print("Classification Report:\n")
print(report)
